[View in Colaboratory](https://colab.research.google.com/github/StefOe/all-conv-pytorch/blob/master/cifar10.ipynb)

In [0]:
cuda = True
train_batch_size = 32
test_batch_size = 124
best_loss = float("inf")
best_epoch = -1
dataset_path = './cifar10'
gsync_save = True

In [0]:
try:
    import torch
except ModuleNotFoundError:
    from os import path
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

    accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.1-{platform}-linux_x86_64.whl
    import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [0]:
try:
    import torchvision
except ModuleNotFoundError:
    !pip install -q torchvision

from torchvision import datasets, transforms

In [0]:
if gsync_save:
    try:
        import utils
    except ModuleNotFoundError:
        !wget https://raw.githubusercontent.com/StefOe/colab-pytorch-utils/HEAD/utils.py
        import utils

    gsync = utils.GDriveSync()

In [0]:
try:
    from allconv import AllConvNet
except ModuleNotFoundError: 
    !wget https://github.com/StefOe/all-conv-pytorch/raw/HEAD/allconv.py
    from allconv import AllConvNet

In [14]:
cuda = cuda and torch.cuda.is_available()
trainset = datasets.CIFAR10(root=dataset_path, train=True, download=True)
train_mean = trainset.train_data.mean(axis=(0,1,2))/255  # [0.49139968  0.48215841  0.44653091]
train_std = trainset.train_data.std(axis=(0,1,2))/255  # [0.24703223  0.24348513  0.26158784]
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(train_mean, train_std),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(train_mean, train_std),
])
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
train_loader = torch.utils.data.DataLoader(datasets.CIFAR10(
    root=dataset_path, train=True, download=True,
    transform=transform_train),
    batch_size=train_batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(root=dataset_path, train=False, download=True,
    transform=transform_test),
    batch_size=test_batch_size, shuffle=False, **kwargs)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [0]:
model = AllConvNet(3)
if cuda:
    model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = optim.lr_scheduler.MultiStepLR(
    optimizer, milestones=[200, 250, 300], gamma=0.1)

In [0]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))
            
def test(epoch, best_loss, best_epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)

        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).data[0]
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(
        '\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset), 100. * correct /
            len(test_loader.dataset)))
    
    if test_loss < best_loss:
        best_epoch = epoch
        best_loss = test_loss
        torch.save(model, "best.pt")
        if gsync_save:
            gsync.update_file_to_folder("best.pt")
    return best_loss, best_epoch

In [0]:
for epoch in range(350):
    scheduler.step()
    train(epoch)
    best_loss, best_epoch = test(epoch, best_loss, best_epoch)

Train Epoch: 0 [0/50000 (0%)]	Loss: 2.301202
Train Epoch: 0 [3200/50000 (6%)]	Loss: 2.302434
Train Epoch: 0 [6400/50000 (13%)]	Loss: 2.299673
Train Epoch: 0 [9600/50000 (19%)]	Loss: 2.307980
Train Epoch: 0 [12800/50000 (26%)]	Loss: 2.298484
Train Epoch: 0 [16000/50000 (32%)]	Loss: 2.302179
Train Epoch: 0 [19200/50000 (38%)]	Loss: 2.301268
Train Epoch: 0 [22400/50000 (45%)]	Loss: 2.304570
Train Epoch: 0 [25600/50000 (51%)]	Loss: 2.300430
Train Epoch: 0 [28800/50000 (58%)]	Loss: 2.303029
Train Epoch: 0 [32000/50000 (64%)]	Loss: 2.303145
Train Epoch: 0 [35200/50000 (70%)]	Loss: 2.302651
Train Epoch: 0 [38400/50000 (77%)]	Loss: 2.305348
Train Epoch: 0 [41600/50000 (83%)]	Loss: 2.301529
Train Epoch: 0 [44800/50000 (90%)]	Loss: 2.303548
Train Epoch: 0 [48000/50000 (96%)]	Loss: 2.304010

Test set: Average loss: 0.0187, Accuracy: 1000/10000 (10%)



Uploading file best.pt: 100%|██████████| 100/100 [00:00<00:00, 144.70it/s]


Train Epoch: 1 [0/50000 (0%)]	Loss: 2.304670
Train Epoch: 1 [3200/50000 (6%)]	Loss: 2.302471
Train Epoch: 1 [6400/50000 (13%)]	Loss: 2.301822
Train Epoch: 1 [9600/50000 (19%)]	Loss: 2.306013
Train Epoch: 1 [12800/50000 (26%)]	Loss: 2.301746
Train Epoch: 1 [16000/50000 (32%)]	Loss: 2.300900
Train Epoch: 1 [19200/50000 (38%)]	Loss: 2.306535
Train Epoch: 1 [22400/50000 (45%)]	Loss: 2.301388
Train Epoch: 1 [25600/50000 (51%)]	Loss: 2.305327
Train Epoch: 1 [28800/50000 (58%)]	Loss: 2.303864
Train Epoch: 1 [32000/50000 (64%)]	Loss: 2.302201
Train Epoch: 1 [35200/50000 (70%)]	Loss: 2.302491
Train Epoch: 1 [38400/50000 (77%)]	Loss: 2.302718
Train Epoch: 1 [41600/50000 (83%)]	Loss: 2.303147
Train Epoch: 1 [44800/50000 (90%)]	Loss: 2.302729
Train Epoch: 1 [48000/50000 (96%)]	Loss: 2.302319

Test set: Average loss: 0.0187, Accuracy: 1000/10000 (10%)



Uploading file best.pt: 100%|██████████| 100/100 [00:00<00:00, 149.18it/s]


Train Epoch: 2 [0/50000 (0%)]	Loss: 2.302569
Train Epoch: 2 [3200/50000 (6%)]	Loss: 2.302565
Train Epoch: 2 [6400/50000 (13%)]	Loss: 2.302988
Train Epoch: 2 [9600/50000 (19%)]	Loss: 2.302354
Train Epoch: 2 [12800/50000 (26%)]	Loss: 2.301873
Train Epoch: 2 [16000/50000 (32%)]	Loss: 2.303240
Train Epoch: 2 [19200/50000 (38%)]	Loss: 2.302656
Train Epoch: 2 [22400/50000 (45%)]	Loss: 2.302915
Train Epoch: 2 [25600/50000 (51%)]	Loss: 2.300892
Train Epoch: 2 [28800/50000 (58%)]	Loss: 2.302483
Train Epoch: 2 [32000/50000 (64%)]	Loss: 2.302700
Train Epoch: 2 [35200/50000 (70%)]	Loss: 2.302490
Train Epoch: 2 [38400/50000 (77%)]	Loss: 2.302285
Train Epoch: 2 [41600/50000 (83%)]	Loss: 2.301330
Train Epoch: 2 [44800/50000 (90%)]	Loss: 2.302984
Train Epoch: 2 [48000/50000 (96%)]	Loss: 2.302592

Test set: Average loss: 0.0187, Accuracy: 1000/10000 (10%)



Uploading file best.pt: 100%|██████████| 100/100 [00:00<00:00, 150.71it/s]


Train Epoch: 3 [0/50000 (0%)]	Loss: 2.302512
Train Epoch: 3 [3200/50000 (6%)]	Loss: 2.302413
Train Epoch: 3 [6400/50000 (13%)]	Loss: 2.302568
Train Epoch: 3 [9600/50000 (19%)]	Loss: 2.303024
Train Epoch: 3 [12800/50000 (26%)]	Loss: 2.302859
Train Epoch: 3 [16000/50000 (32%)]	Loss: 2.302206
Train Epoch: 3 [19200/50000 (38%)]	Loss: 2.302402
Train Epoch: 3 [22400/50000 (45%)]	Loss: 2.302208
Train Epoch: 3 [25600/50000 (51%)]	Loss: 2.302541
Train Epoch: 3 [28800/50000 (58%)]	Loss: 2.302618
Train Epoch: 3 [32000/50000 (64%)]	Loss: 2.302905
Train Epoch: 3 [35200/50000 (70%)]	Loss: 2.302374
Train Epoch: 3 [38400/50000 (77%)]	Loss: 2.302583
Train Epoch: 3 [41600/50000 (83%)]	Loss: 2.302378
Train Epoch: 3 [44800/50000 (90%)]	Loss: 2.302264
Train Epoch: 3 [48000/50000 (96%)]	Loss: 2.302325

Test set: Average loss: 0.0187, Accuracy: 1000/10000 (10%)



Uploading file best.pt: 100%|██████████| 100/100 [00:00<00:00, 167.22it/s]


Train Epoch: 4 [0/50000 (0%)]	Loss: 2.303740
Train Epoch: 4 [3200/50000 (6%)]	Loss: 2.302080
Train Epoch: 4 [6400/50000 (13%)]	Loss: 2.302977
Train Epoch: 4 [9600/50000 (19%)]	Loss: 2.302250
Train Epoch: 4 [12800/50000 (26%)]	Loss: 2.302896


In [0]:
# if in Google Colab, download your model with this
from google.colab import files
files.download("best.pt")